## 1.1) MakeNcRNAmatrix

    Input : fasta format
    Output : ./outdir

## 1.2) AssemblyMatrix
    
    Input : ./outdir
    Output : "ncRNApair_data.npy" And "ncRNApair_labe.npy"

## 2) DeepLearning

    Input : "ncRNApair_data.npy" And "ncRNApair_labe.npy"
    Output :

In [5]:
!pip3 install biopython

You should consider upgrading via the 'c:\users\samsa\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


## MakeNcRNAmatrix

    Input : fasta format
    Output : ./outdir

In [5]:
import sys
import subprocess
import re

from Bio import SeqIO
import random
 
import numpy as np


def run_DAFS(path):
    inn = subprocess.Popen(['dafscnn', path], stdout=subprocess.PIPE)
    buf = []
    while True:
        innline = inn.stdout.readline().decode('utf-8')
        buf.append(innline)
        
        if not innline:
            break

    return buf[4].strip().upper(), buf[6].strip().upper()

def get_basepair_matrix(path_to_bpfile):
    
    mat = []
    lineArray =[]
    seqlen = []
    l = 0
    ## Store data in list(mat) from input file.
    for line in open(path_to_bpfile,'r'):
        if line[0] == '>':
            mat = mat + [lineArray]
            seqlen = seqlen + [l]
            
            lineArray = []
            l = 0
        else:
            li = re.split('[\s]', line)
            li.pop()
            lineArray = lineArray + [li]
            l += 1
    mat = mat + [lineArray]
    seqlen = seqlen + [l]
    mat.pop(0)
    seqlen.pop(0)
    
    i = 0
    Array = []
    ## Make base-pair probability matrix(Array) from list(mat)
    for n in seqlen:
        arr = np.zeros((n,n), float)
        for j in range(n):
            for k in mat[i][j]:
                rate = re.split('[:]', k)
                if len(rate) > 1:
                    arr[j][int(rate[0])] = float(rate[1])
        i += 1
        Array = Array + [arr]

    bp_mat = []
    ## Calcurate base-pair probability
    for i in range(len(seqlen)):
        bp = np.zeros([seqlen[i], 3], dtype=np.float32)
        # each base
        for j in range(seqlen[i]):
            Left = 0
            Right = 0
            # probability of (
            for k in range(j, seqlen[i]):
                Left += Array[i][j][k]
            # probability of )
            for k in range(j):
                Right += Array[i][k][j]
            bp[j][0] = Left
            bp[j][1] = Right
            bp[j][2] = 1 - Left - Right
            
        bp_mat = bp_mat + [bp]
    #print(bp_mat)

    return bp_mat

def make_pairFASTA(dataset, itr1, outpath):
    train_data = np.empty((0,1200,16), dtype=np.float32)
    train_label = np.empty((0), dtype=np.int32)

    for j in range(itr1+1, len(dataset)):
        # make pairFASTA file
        path_to_pairFasta = "./pair" + str(itr1) + "," + str(j) + ".fa"
        f = open(path_to_pairFasta, 'w')
        pairFa = ""
        for k in [dataset[itr1], dataset[j]]:
            pairFa += ">" + k[0] + "\n" + k[2] + "\n"
        f.write(pairFa)
        f.close()

        # calculate pairwise alignment score by DAFS
        pair1, pair2 = run_DAFS(path_to_pairFasta)
        subprocess.call(["rm", path_to_pairFasta])

        # get base pair matrix
        path_to_bpfile = "bp" + dataset[itr1][0] + "_" + dataset[j][0] + ".txt"
        bp_mat = get_basepair_matrix(path_to_bpfile)
        subprocess.call(["rm", path_to_bpfile])

        # make train data
        data = np.zeros(([1200,16]), dtype=np.float32)
        n1 = 0 
        n2 = 0 
        for k in range(len(pair1)):
            ## pair1's data
             # bases data
            if pair1[k] == "A":
                data[k][0] = 1 
            elif pair1[k] == "T":
                data[k][1] = 1 
            elif pair1[k] == "G":
                data[k][2] = 1 
            elif pair1[k] == "C":
                data[k][3] = 1 
            elif pair1[k] == "-":
                data[k][4] = 1 
             # base-pair probability data
            if pair1[k] != "-":
                data[k][5] = bp_mat[0][n1][0]
                data[k][6] = bp_mat[0][n1][1]
                data[k][7] = bp_mat[0][n1][2]
                n1 += 1
            
            ## pair2's data
             # bases data
            if pair2[k] == "A":
                data[k][8] = 1 
            elif pair2[k] == "T":
                data[k][9] = 1 
            elif pair2[k] == "G":
                data[k][10] = 1 
            elif pair2[k] == "C":
                data[k][11] = 1 
            elif pair2[k] == "-":
                data[k][12] = 1 
            # base-pair probability data
            if pair2[k] != "-":
                data[k][13] = bp_mat[1][n2][0]
                data[k][14] = bp_mat[1][n2][1]
                data[k][15] = bp_mat[1][n2][2]
                n2 += 1

        # make train label
        if dataset[itr1][1] == dataset[j][1]:
            label = 0
        else:
            label = 1

        train_data = np.append(train_data, np.array([data]), axis=0)
        train_label = np.append(train_label, label)
            
    #print(train_data)
    #print(train_label)

    outdata = outpath + "/portion/ncRNApair_data" + str(itr1) + ".npy"
    outlabel = outpath + "/portion/ncRNApair_label" + str(itr1) + ".npy"    

    np.save(outdata, train_data)
    np.save(outlabel, train_label)


if __name__ == '__main__':

    infile = sys.argv[1]
    itr1 = 1
    outpath = './outdir'

    if outpath[-1] == "/":
        outpath = outpath[:-1]

    ###### read fasta file #######
    dataset = []
    out = ""
    for record in SeqIO.parse(infile, "fasta"):
        id_part = record.id
        id_parts = id_part.split(",")
        seq_part = str(record.seq.upper())

        # geneset : [[gene name, genelabel(mi=0,sno=1,t=2), sequence],...
        dataset = dataset + [[id_parts[0], int(id_parts[1]), seq_part]]
        
        out += id_parts[0] + ":" + id_parts[1] + "\n"
    f = open(outpath+"/genelabel.txt", "w")
    f.write(out)
    f.close()

    ##############################

    make_pairFASTA(dataset, itr1, outpath)

FileNotFoundError: [Errno 2] No such file or directory: '-f'

## AssemblyMatrix
    Input : ./outdir
    Output : "ncRNApair_data.npy" And "ncRNApair_labe.npy"

In [6]:
def assembledata(ddir):

    checkdata = np.load(ddir+"/portion/ncRNApair_data0.npy")
    dsize = len(checkdata) + 1
    print('dsize =',dsize)
    genelen = len(checkdata[0])
    width = len(checkdata[0][0])

    data = np.empty((0,genelen,width), dtype=np.float32)
    label = np.empty((0), dtype=np.int32)

    print("Makeing data...")
    start = time.time()

    for i in range(2):
        indata = ddir + "/portion/ncRNApair_data"+str(i)+".npy"
        inlabel = ddir + "/portion/ncRNApair_label"+str(i)+".npy"

        tmpdata = np.load(indata)
        tmplabel = np.load(inlabel)
        
        print(tmpdata.shape[0])

        data = np.concatenate([data, tmpdata], axis=0)
        label = np.concatenate([label, tmplabel], axis=0)

    print("Complete makeing data.")
    elapsed_time = time.time() - start
    print ("Loding time:{0}".format(elapsed_time) + "[sec]")
    print("Data size :",len(data),"*",len(data[0]),"*",len(data[0][0]))

    print('here :',ddir+"/ncRNApair_data.npy")
    np.save("ncRNApair_data.npy", data)
    np.save("ncRNApair_label.npy", label)


In [3]:
temo =  np.load("ncRNApair_label.npy")

In [4]:
temo

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [8]:
datadir = sys.argv[1]
if datadir[-1] == "/":
    datadir = datadir[:-1]

assembledata('./outdir')

dsize = 60
Makeing data...
59
58
Complete makeing data.
Loding time:0.018949508666992188[sec]
Data size : 117 * 1200 * 16
here : ./outdir/ncRNApair_data.npy


In [37]:
genelabellist = []
for line in open('./outdir/genelabel.txt', 'r'):
    lines = line.strip().split(":")    # Separater is ":".
    genelabellist.append(int(lines[1]))
    
genelabellist = np.asarray(genelabellist)
inputfam = np.unique(genelabellist)

## DeepLearning
    Input : "ncRNApair_data.npy" And "ncRNApair_labe.npy"
    Output : 

In [10]:
x_test = np.load('ncRNApair_data.npy')
y_test = np.load('ncRNApair_label.npy')

In [40]:
x_train = np.load('testdata/ncRNApairdataDAFS_test.npy')
y_train = np.load('testdata/ncRNApairlabel_test.npy')

In [41]:
print('x_train.shape =',x_train.shape)
print('x_test.shape =',x_test.shape)

x_train.shape = (1770, 1200, 16)
x_test.shape = (117, 1200, 16)


In [42]:
import tensorflow as tf
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(1200,16)))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=16, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.4))
cnn.add(tf.keras.layers.Dense(units=8, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.3))

In [43]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [44]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [45]:
cnn.fit(x = x_train, y = y_train, epochs = 5)

Epoch 1/5
56/56 [==============================] - 1s 18ms/step - loss: 0.7593 - accuracy: 0.5463
Epoch 2/5
56/56 [==============================] - 1s 18ms/step - loss: 0.6428 - accuracy: 0.6373
Epoch 3/5
56/56 [==============================] - 1s 18ms/step - loss: 0.5892 - accuracy: 0.6927
Epoch 4/5
56/56 [==============================] - 1s 18ms/step - loss: 0.5077 - accuracy: 0.7814
Epoch 5/5
56/56 [==============================] - 1s 20ms/step - loss: 0.4582 - accuracy: 0.8175


In [51]:
from sklearn.metrics import confusion_matrix, accuracy_score

def get_res(y_pred):
    res_test = []
    for i in y_pred:
        if i<0.5: res_test.append(0)
        else:
            res_test.append(1)
    return res_test

y_pred_train = cnn.predict(x_train)
y_pred_train = y_pred_train.reshape((1770,))

res = get_res(y_pred_train)
confusion_matrix(y_train, res)
print('Accuracy for predicting training set :',accuracy_score(res,y_train))

Accuracy for predicting training set : 0.867231638418079


In [53]:
y_test_pred  = cnn.predict(x_test)
y_test_pred = y_test_pred.reshape((117,))
res = get_res(y_test_pred)
confusion_matrix(y_test, res)
print('Accuracy for predicting test set :',accuracy_score(res,y_test))

Accuracy for predicting test set : 0.8632478632478633
